In [2]:
import pathlib

import pandas as pd
import matplotlib as mpl
from matplotlib import pyplot as plt
from pandas_profiling import ProfileReport

%matplotlib inline

mpl.rcParams["figure.figsize"] = (10, 10)
mpl.rcParams["figure.dpi"] = 100
plt.style.use('seaborn')

In [3]:
%load_ext autoreload
%autoreload 2

from notebook_utils import log_transform, inverse_log_transform

In [4]:
data_path = pathlib.Path("data", "raw", "TOP500_202106.csv")

In [5]:
data = pd.read_csv(data_path, engine="c", index_col="Rank")

In [6]:
data.drop(["Previous Rank", "First Appearance", "First Rank"], inplace=True, axis="columns")

In [13]:
data.i

ValueError: invalid literal for int() with base 10: '7,630,848'

In [7]:
data.describe()

,Year,Power Efficiency [GFlops/Watts],Cores per Socket,System ID
count,500.000000,181.000000,500.00000,500.000000
mean,2018.624000,6.158508,24.45200,179564.112000
std,1.619837,6.601645,17.74761,429.311552
min,2010.000000,0.190000,4.00000,176908.000000
25%,2018.000000,2.250000,18.00000,179448.750000
50%,2019.000000,3.650000,20.00000,179695.500000
75%,2020.000000,6.390000,24.00000,179835.250000
max,2021.000000,29.700000,260.00000,179974.000000


In [8]:
data.head(n=5)

,Name,Computer,Site,Manufacturer,Country,Year,Segment,Total Cores,Accelerator/Co-Processor Cores,Rmax [TFlop/s],...,Accelerator/Co-Processor,Cores per Socket,Processor Generation,System Model,System Family,Interconnect Family,Interconnect,Continent,Site ID,System ID
Rank,,,,,,,,,,,,,,,,,,,,,
1,Supercomputer Fugaku,"Supercomputer Fugaku, A64FX 48C 2.2GHz, Tofu i...",RIKEN Center for Computational Science,Fujitsu,Japan,2020,Research,"7,630,848",NaN,"442,010.00",...,None,48,Fujitsu A64FX,Supercomputer Fugaku,Fujitsu Cluster,Proprietary Network,Tofu interconnect D,Asia,"50,831",179807
2,Summit,"IBM Power System AC922, IBM POWER9 22C 3.07GHz...",DOE/SC/Oak Ridge National Laboratory,IBM,United States,2018,Research,"2,414,592","2,211,840","148,600.00",...,NVIDIA Volta GV100,22,IBM POWER9,IBM Power System AC922,IBM Power Systems,Infiniband,Dual-rail Mellanox EDR Infiniband,North America,"48,553",179397
3,Sierra,"IBM Power System AC922, IBM POWER9 22C 3.1GHz,...",DOE/NNSA/LLNL,IBM / NVIDIA / Mellanox,United States,2018,Research,"1,572,480","1,382,400","94,640.00",...,NVIDIA Volta GV100,22,IBM POWER9,IBM Power System AC922,IBM Power Systems,Infiniband,Dual-rail Mellanox EDR Infiniband,North America,"49,763",179398
4,Sunway TaihuLight,"Sunway MPP, Sunway SW26010 260C 1.45GHz, Sunway",National Supercomputing Center in Wuxi,NRCPC,China,2016,Research,"10,649,600",NaN,"93,014.59",...,None,260,Sunway,Sunway MPP,Sunway Cluster,Custom Interconnect,Sunway,Asia,"50,623",178764
5,Perlmutter,"HPE Cray EX235n, AMD EPYC 7763 64C 2.45GHz, NV...",DOE/SC/LBNL/NERSC,HPE,United States,2021,Research,"706,304","615,168","64,590.00",...,NVIDIA A100 SXM4 40 GB,64,AMD Milan,HPE Cray EX235n,HPE Cray EX,Gigabit Ethernet,Slingshot-10,North America,"48,429",179972


In [9]:
profile = ProfileReport(data, title="Pandas Profiling Report")

In [10]:
profile.to_file("report.html")

Summarize dataset:  83%|████████▎ | 40/48 [00:04<00:00, 10.06it/s, Calculate cramers correlation]D:\AnacondaEnvs\made-hpc-hw1-1\lib\site-packages\pandas_profiling\model\correlations.py:146: UserWarning: There was an attempt to calculate the cramers correlation, but this failed.
To hide this warning, disable the calculation
(using `df.profile_report(correlations={"cramers": {"calculate": False}})`
If this is problematic for your use case, please report this as an issue:
https://github.com/pandas-profiling/pandas-profiling/issues
(include the error message: 'No data; `observed` has size 0.')
  warnings.warn(
Export report to file: 100%|██████████| 1/1 [00:00<00:00, 76.94it/s]
